## Imports

In [ ]:
import pandas as pd
import numpy as np
import calendar
import holidays
from datetime import datetime

date_format: str = "%m/%d/%YT%H:%M:%SZ"
date_format_weather: str = "%Y-%m-%dT%H:%M:%S"
us_holidays = holidays.UnitedStates()

## Helper Functions

In [ ]:
# mode of series
def mode_(s):
    try:
        return s.mode()[0]
    except IndexError:
        return np.nan

# convert string to datetime
def convert_to_datetime_string(row):
    try:
        date_arr = row['INCIDENT_DATE'].split('/')
        time_arr = row['INCIDENT_TIME'].split(':')
        month = int(date_arr[0])
        day = int(date_arr[1])
        year = int(date_arr[2])
        hour = int(time_arr[0])
        datetime_str = datetime(year, month, day, hour, 0).strftime(date_format)
        return datetime_str
    except:
        return "NULL"

# prepare datetime for category
def generate_predictors(df):
    df['year'] = df['datetime'].apply(lambda x: x.year)
    df['day_of_week'] = df['datetime'].apply(lambda x: x.isoweekday())
    df['day_of_month'] = df['datetime'].apply(lambda x: x.day)
    df['hour_of_day'] = df['datetime'].apply(lambda x: x.hour)
    df['day_of_year'] = df['datetime'].apply(lambda x: x.timetuple().tm_yday)
    df['month_of_year'] = df['datetime'].apply(lambda x: x.month)
    df['is_holiday'] = df['datetime'].apply(lambda x: x in us_holidays)
    df['is_leap_year'] = df['year'].apply(lambda x: calendar.isleap(x))
    return df

## Aggregate Data

In [ ]:
calls = pd.read_csv('../data/nypd_911.csv')

# reduce features
calls = calls[["INCIDENT_TIME", "INCIDENT_DATE", "CAD_EVNT_ID", "BORO_NM"]]

# convert time to datetime string and drop extra columns
calls['time'] = calls.apply(convert_to_datetime_string, axis=1)
calls = calls.drop(['INCIDENT_DATE', 'INCIDENT_TIME'], axis=1)

# drop null time entries
calls = calls[calls['time'] != 'NULL']

# apply datetime conversion
calls['datetime'] = pd.to_datetime(calls['time'], format=date_format)
calls = calls.drop(['time'], axis=1)

# generate datetime category columns
generate_predictors(calls)
calls = calls.drop(['datetime'], axis=1)

# group hourly entries by borough and count number of incidents
gb_index = ['BORO_NM', 'year', "month_of_year", "day_of_month", "day_of_week", "day_of_year", "hour_of_day", "is_holiday", "is_leap_year"]
grouped_calls = pd.DataFrame(calls.groupby(gb_index).agg({"CAD_EVNT_ID": "count"})).reset_index()

# delete duplicates and remove entries with null borough
grouped_calls.drop_duplicates(subset=gb_index, keep='first', inplace=True, ignore_index=True)
final_df = grouped_calls[grouped_calls['BORO_NM'] != '(null)']
final_df = final_df.sort_values(gb_index)

# rename target: number of calls
final_df['target'] = final_df['CAD_EVNT_ID'] 
final_df = final_df.drop('CAD_EVNT_ID', axis=1)

# drop any null entries
final_df = final_df.dropna()

# sanity check
print("\nMissing Values:")
print(final_df.isna().sum())

# save to csv
final_df.to_csv('../data/nyc_grouped_incidents.csv', index=False)